In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import copy
import sys
import xarray as xr
import numpy as np
import dask.array as da

import matplotlib.pyplot as plt
import hvplot.xarray
import scipy.constants

sys.path.append("..")
import processing_dask as pr
import plot_dask
import processing as old_processing

import colorednoise as cn

sys.path.append("../../preprocessing/")
#from generate_chirp import generate_chirp

sys.path.append("..")
from processing import pulse_compress

In [ ]:
def generate_chirp_with_jitter(config, t_jitter=0):
    """
    Generate a chirp according to parameters in the config dictionary, typically
    loaded from a config YAML file.

    Returns a tuple (ts, chirp_complex), where ts is a numpy array of time
    samples, and chirp_complex is a numpy array of complex floating point values
    representing the chirp.

    If you're looking for a floating point valued chirp to use in convolution,
    this is probably the right function.

    This function does not convert the complex numpy array to the cpu format
    expected by the radar code. If you want to produce samples to feed the radar
    code, look at `generate_from_yaml_filename` (later in this file) instead.
    """
    # Load parameters
    gen_params = config["GENERATE"]
    chirp_type = gen_params["chirp_type"]
    sample_rate = gen_params["sample_rate"]
    chirp_bandwidth = gen_params["chirp_bandwidth"]
    offset = gen_params.get("lo_offset_sw", 0)
    window = gen_params["window"]
    chirp_length = gen_params["chirp_length"]
    pulse_length = gen_params.get("pulse_length", chirp_length) # default to chirp_length is no pulse_length is specified

    # Build chirp

    end_freq = chirp_bandwidth / 2 # Chirp goes from -BW/2 to BW/2
    start_freq = -1 * end_freq

    start_freq += offset
    end_freq += offset

    ts = np.arange(0, chirp_length-(1/(2*sample_rate)), 1/(sample_rate)) + t_jitter
    ts_zp = np.arange(0, (pulse_length)-(1/(2*sample_rate)), 1/(sample_rate))

    if chirp_type == 'linear':
        ph = 2*np.pi*((start_freq)*ts + (end_freq - start_freq) * ts**2 / (2*chirp_length))
    elif chirp_type == 'hyperbolic':
        ph = 2*np.pi*(-1*start_freq*end_freq*chirp_length/(end_freq-start_freq))*np.log(1- (end_freq-start_freq)*ts/(end_freq*chirp_length))
    else:
        ph = 2*np.pi*(start_freq*ts + (end_freq - start_freq) * ts**2 / (2*chirp_length))
        printf("[ERROR] Unrecognized chirp type '{chirp_type}'")
        return None, None

    chirp_complex = np.exp(1j*ph)

    if window == "blackman":
        chirp_complex = chirp_complex * np.blackman(chirp_complex.size)
    elif window == "hamming":
        chirp_complex = chirp_complex * np.hamming(chirp_complex.size)
    elif window == "kaiser14":
        chirp_complex = chirp_complex * np.kaiser(chirp_complex.size, 14.0)
    elif window == "kaiser10":
        chirp_complex = chirp_complex * np.kaiser(chirp_complex.size, 10.0)
    elif window == "kaiser18": 
        chirp_complex = chirp_complex * np.kaiser(chirp_complex.size, 18.0)
    elif window != "rectangular":
        print("[ERROR] Unrecognized window function '{window}'")
        return None, None

    chirp_complex = np.pad(chirp_complex, (int(np.floor(ts_zp.size - ts.size)/2),), 'constant')

    chirp_complex = chirp_complex

    return ts_zp, chirp_complex


In [ ]:
def plot_signal(ts, sig, sample_rate, one_sided=False, log_scale=False, nperseg=2**10, ax1=None, ax2=None, label="", alpha=1.0, time_delay=None):
    if ax1 is None or ax2 is None:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        fig.tight_layout()
    else:
        fig = None

    # Plot in the time domain
    ax1.plot(ts * 1e6, np.real(sig), label=label+" [real]", alpha=alpha)
    if not one_sided:
        ax1.plot(ts * 1e6, np.imag(sig), label=label+" [imag]", alpha=alpha)
    ax1.set_xlabel('Time [us]')
    ax1.set_ylabel('Amplitude')
    ax1.set_title('Time Domain')
    ax1.grid(True)

    # Plot in the frequency domain
    freq, spectrum = scipy.signal.welch(sig, fs=sample_rate, nperseg=nperseg, return_onesided=one_sided, detrend=False, window='rectangular', scaling='density')
    
    if time_delay is not None:
        ratio = 4 * (np.sin(np.pi * freq * time_delay))**2
    
    if not one_sided:
        freq = np.fft.fftshift(freq)
        spectrum = np.fft.fftshift(spectrum)
    #freq = np.fft.fftshift(np.fft.fftfreq(len(sig), d=1/sample_rate))
    #spectrum = np.fft.fftshift(np.fft.fft(sig))
    ax2.plot(freq / 1e6, 10*np.log10(np.abs(spectrum)), label=label, alpha=alpha)

    if time_delay is not None:
        ax2.plot(freq / 1e6, 10*np.log10(np.abs(ratio*spectrum)), label=f"Theoretical for delay {time_delay}", alpha=alpha, linestyle='--')

    if log_scale:
        ax2.set_xscale('log')
        ax2.set_xlim([0.01, None])
        if not one_sided:
            raise ValueError("Can't use log scale with two-sided spectrum")
    ax2.set_xlabel('Frequency [MHz]')
    ax2.set_ylabel('Power [dB]')
    ax2.set_title('Frequency Domain')
    ax2.grid(True)

    return fig, (ax1, ax2)


In [ ]:
fs = 10e6
ts = np.arange(0, 1.0, 1/fs)

# f_minus2 = 10**(-75/20) * cn.powerlaw_psd_gaussian(2, len(ts))
# f_minus1 = 10**(-95/20) * cn.powerlaw_psd_gaussian(1, len(ts))
# f_zero = 10**(-80/20) * cn.powerlaw_psd_gaussian(0, len(ts))
#ph_noise_ref_osc = 10**(60/20) * (f_minus2 + f_minus1 + f_zero)

f_zero = 10**(-80/20) * cn.powerlaw_psd_gaussian(0, len(ts))
f_zero_2 = 10**(10/20) * cn.powerlaw_psd_gaussian(0, len(ts))
# bandpass f_zero_2 with a 1 kHz bandpass filter
sos = scipy.signal.butter(1, [1, 10], btype='bandpass', fs=fs, output='sos')
f_zero_2_filt = scipy.signal.sosfilt(sos, f_zero_2)
ph_noise_ref_osc = f_zero_2_filt + f_zero

ph_noise_clock = (fs/40e6) * ph_noise_ref_osc
ph_noise_lo = (400e6/40e6) * ph_noise_ref_osc

In [ ]:
nperseg = 2**17
print(len(ts) / nperseg)
fig, (ax1, ax2) = plot_signal(ts, ph_noise_ref_osc, sample_rate=fs, one_sided=True, log_scale=True, nperseg=nperseg, label="sum")
#plot_signal(ts, f_minus2, sample_rate=fs, one_sided=True, log_scale=True, nperseg=nperseg, ax1=ax1, ax2=ax2, label="f^-2")
#plot_signal(ts, f_minus1, sample_rate=fs, one_sided=True, log_scale=True, nperseg=nperseg, ax1=ax1, ax2=ax2, label="f^-1")
#plot_signal(ts, f_zero, sample_rate=fs, one_sided=True, log_scale=True, nperseg=nperseg, ax1=ax1, ax2=ax2, label="f^0")

ax1.legend(loc="upper right"), ax2.legend(loc="upper right")
ax2.set_xlim([0.0001, 5])
ax2.set_xticks([0.0001, 0.001, 0.01, 0.1, 1, 5])
ax2.set_xticklabels(["100 Hz", "1 kHz", "10 kHz", "100 kHz", "1 MHz", "5 MHz"])
ax2.set_ylim([-160, -50])

ax2.scatter([0.0001, 0.001, 0.01, 0.1, 1, 5], np.array([-117, -137, -149,-151,-151,-151])+3, color='black', marker='x', zorder=10, label="Reference")

fig.tight_layout()

In [ ]:
reflection_distance = 2800 #0 # m (one-way)
reflection_delay_time = 2 * reflection_distance / (scipy.constants.c / np.sqrt(3.17))
reflection_delay_samples = int(fs * reflection_delay_time)

In [ ]:
t_jitter = (ph_noise_clock[:-reflection_delay_samples] - ph_noise_clock[reflection_delay_samples:]) / (2 * np.pi * fs)

In [ ]:
fix, (ax1, ax2) = plot_signal(ts[:-reflection_delay_samples], (ph_noise_clock[:-reflection_delay_samples] - ph_noise_clock[reflection_delay_samples:]), sample_rate=fs, one_sided=True, log_scale=True, nperseg=nperseg, label="ph_noise_clock (cancelled)")
plot_signal(ts[:-reflection_delay_samples], ph_noise_clock[:-reflection_delay_samples], sample_rate=fs, one_sided=True, log_scale=True, nperseg=nperseg, label="ph_noise_clock (one-way)", ax1=ax1, ax2=ax2)
plot_signal(ts[:-reflection_delay_samples], ph_noise_lo[:-reflection_delay_samples] - ph_noise_lo[reflection_delay_samples:], sample_rate=fs, one_sided=True, log_scale=True, nperseg=nperseg, ax1=ax1, ax2=ax2, label="ph_noise_lo (cancelled)")
plot_signal(ts[:-reflection_delay_samples], ph_noise_lo[:-reflection_delay_samples], sample_rate=fs, one_sided=True, log_scale=True, nperseg=nperseg, ax1=ax1, ax2=ax2, label="ph_noise_lo (one-way)")
ax2.legend()
ax2.set_xlim([0.0001, 5])
ax2.set_xticks([0.0001, 0.001, 0.01, 0.1, 1, 5])
ax2.set_xticklabels(["100 Hz", "1 kHz", "10 kHz", "100 kHz", "1 MHz", "5 MHz"])
fig.tight_layout()

In [ ]:
config = {
    'GENERATE':
        {
            'chirp_type': 'linear',
            'sample_rate': fs,
            'chirp_bandwidth': 5e6,
            'window': "rectangular",
            'chirp_length': 20e-6,
            'pulse_length': 20e-6
        }
}
expected_chirp_length = len(np.arange(0, (config['GENERATE']['chirp_length'])-(1/(2*config['GENERATE']['sample_rate'])), 1/(config['GENERATE']['sample_rate'])))

pri = 50e-6
n_chirps = int((len(ts)-reflection_delay_samples-1) // (pri * fs))
chirps = np.zeros(shape=(n_chirps, int(fs*pri)), dtype=np.complex64)
ts_chirp, reference_chirp = generate_chirp_with_jitter(config)
print(f"n_chirps: {n_chirps}, prf: {1/pri} Hz")

delay_samples = 200

for i in range(n_chirps):
    jitter_start_idx = int(i * (pri * fs))
    # Generate chirps with jittered timestamps, simulating effects of DAC and ADC clock phase noise
    ts_chirp, chirps[i, delay_samples:delay_samples+expected_chirp_length] = generate_chirp_with_jitter(config)#, t_jitter=t_jitter[jitter_start_idx:jitter_start_idx+expected_chirp_length])
    # Add in phase contributions from TX and RX LOs
    chirps[i, delay_samples:delay_samples+expected_chirp_length] *= np.exp(1j *
                                                                           (ph_noise_lo[jitter_start_idx:jitter_start_idx+expected_chirp_length] -
                                                                            0*ph_noise_lo[jitter_start_idx+reflection_delay_samples:jitter_start_idx+reflection_delay_samples+expected_chirp_length]))


In [ ]:
n_stacks = np.append(np.logspace(0, int(np.log10(n_chirps)), 10*(int(np.log10(n_chirps))+1), dtype=int), int(n_chirps))
peaks_lin_mean = np.zeros_like(n_stacks, dtype=np.float64)
peaks_lin_std = np.zeros_like(n_stacks, dtype=np.float64)

nstack_1_peak_phases = np.zeros(shape=(n_chirps,))

for i, n_stack in enumerate(n_stacks):
    peaks_lin = []
    for start_idx in np.arange(0, n_chirps-n_stack+1, n_stack):
        stacked = np.mean(chirps[start_idx:start_idx+n_stack, :], axis=0)
        fast_time, compressed = pulse_compress(stacked, reference_chirp, fs)
        peaks_lin.append(np.max(np.abs(compressed)))
        if n_stack == 1:
            nstack_1_peak_phases[start_idx] = np.angle(compressed[np.argmax(np.abs(compressed))])
    #print(peaks_lin)
    peaks_lin_mean[i] = np.mean(peaks_lin)
    peaks_lin_std[i] = np.std(peaks_lin)

#integration_time = n_stacks * pri

In [ ]:
from matplotlib.ticker import EngFormatter

effective_prf = 1/(n_stacks * pri)

fig, ax = plt.subplots()
ax.scatter(n_stacks, 20*np.log10(peaks_lin_mean))
ax.set_xscale('log')

ax.set_xscale('log')
#ax.set_xlim(ax.get_xlim())
x_n = np.logspace(0, int(np.log10(n_chirps)), (int(np.log10(n_chirps))+1), dtype=int)
ax.set_xticks(x_n)
ax.set_xlabel("Number of stacked chirps")
ax.set_xlim([1, 1000])

ax_prf = ax.twiny()
formatter0 = EngFormatter(unit='Hz')
#ax_prf.xaxis.set_major_formatter(formatter0)
ax_prf.set_xlabel("Effective Pulse Repetition Frequency")
ax_prf.set_xlim(1/(np.array(ax.get_xlim()) * pri))
ax_prf.set_xscale('log')
x_pri = 1/(x_n * pri)
ax_prf.set_xticks(x_pri)
ax_prf.set_xticklabels([f"{formatter0(x)}" for x in x_pri])

ax.grid()

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(np.arange(len(nstack_1_peak_phases)), nstack_1_peak_phases)
ax.grid()
fig.tight_layout()

In [ ]:
ph_noise_cancelled = ph_noise_sum[:-356] - ph_noise_sum[356:]
fig, (ax1, ax2) = plot_signal(ts, ph_noise_sum, sample_rate=fs, one_sided=True, log_scale=True, nperseg=nperseg, label="sum", time_delay=356/fs)
plot_signal(ts[:-356], ph_noise_cancelled, sample_rate=fs, one_sided=True, log_scale=True, nperseg=nperseg, ax1=ax1, ax2=ax2, label="cancelled")
ax1.legend(), ax2.legend()
ax2.set_xlim([0.0001, 5])
ax2.set_ylim([-160, -100])

ax2.scatter([0.0001, 0.001, 0.01, 0.1, 1, 5], np.array([-117, -137, -149,-151,-151,-151])+3, color='black', marker='x', zorder=10, label="Reference")

fig.tight_layout()

In [ ]:
t_jitter = ph_noise_sum / (2 * np.pi * fs)

In [ ]:
config = {
    'GENERATE':
        {
            'chirp_type': 'linear',
            'sample_rate': fs,
            'chirp_bandwidth': 15e6,
            'window': "rectangular",
            'chirp_length': 20e-6,
            'pulse_length': 20e-6
        }
}

expected_chirp_length = len(np.arange(0, (config['GENERATE']['chirp_length'])-(1/(2*config['GENERATE']['sample_rate'])), 1/(config['GENERATE']['sample_rate'])))

ts_chirp, chirp = generate_chirp_with_jitter(config, t_jitter=t_jitter[:expected_chirp_length])

plot_signal(ts_chirp, chirp, sample_rate=config['GENERATE']['sample_rate'], one_sided=False, label="chirp")

In [ ]:
pri = 50e-6
n_chirps = int(len(ts) // (pri * fs))
chirps = np.zeros_like(chirp, shape=(n_chirps, int(fs*pri)))
ts_chirp = None

delay_samples = 200

for i in range(n_chirps):
    jitter_start_idx = int(i * (pri * fs))
    ts_chirp, chirps[i, delay_samples:delay_samples+len(chirp)] = generate_chirp_with_jitter(config, t_jitter=t_jitter[jitter_start_idx:jitter_start_idx+expected_chirp_length])

In [ ]:
n_stacks = np.array([1, 10, 100, 1000, n_chirps])
peaks_lin = np.zeros_like(n_stacks, dtype=np.float64)

for i, n_stack in enumerate(n_stacks):
    stacked = np.mean(chirps[:n_stack], axis=0)
    fast_time, compressed = pulse_compress(stacked, chirp, fs)
    peaks_lin[i] = np.max(np.abs(compressed))

fig, ax = plt.subplots()
ax.scatter(n_stacks, 20*np.log10(peaks_lin))
ax.set_xscale('log')
ax.grid()
plt.show()

In [ ]:
stacked = np.mean(chirps[:1000,:], axis=0)

fast_time, compressed = pulse_compress(stacked, chirp, fs)

fig, ax = plt.subplots()
ax.plot(fast_time, np.abs(compressed))


In [ ]:
10*np.log10(0.2**2/4)

In [ ]:
example_signal = np.sin(2*np.pi*10e6*ts_chirp)# + 0.2*np.sin(2*np.pi*0.5e6*ts_chirp))
fig, (ax1, ax2) = plot_signal(ts_chirp, example_signal, sample_rate=fs, one_sided=True)
#ax2.set_ylim(-80, -40)

In [ ]:
stacked.shape

In [ ]:
1	-75
10	-110
100	-132
1000	-142
10000	-145
100000	-150


In [ ]:
np.stack([phase_noise_filter_freqs[:-1], phase_noise_filter_freqs[1:]]).T, np.stack([phase_noise_filter_amplitude[:-1], phase_noise_filter_amplitude[1:]]).T

In [ ]:
phase_noise_filter_freqs = np.array(   [0,  10, 100, 1000, config['GENERATE']['sample_rate']/2]) # Hz
phase_noise_filter_power_db = np.array([0, -50, -80, -100, -100]) # dBc/Hz

phase_noise_filter_freqs = np.array(   [0,  1, 10, 100, 1000, config['GENERATE']['sample_rate']/2]) # Hz
phase_noise_filter_power_db = np.array([0, -10, -20, -30, -40, -50]) # dBc/Hz

phase_noise_filter_amplitude = 10**(phase_noise_filter_power_db/20)

#taps = scipy.signal.firwin2(10001, phase_noise_filter_freqs, phase_noise_filter_amplitude, antisymmetric=False, fs=config['GENERATE']['sample_rate'])

taps = scipy.signal.firls(10001,
                          np.stack([phase_noise_filter_freqs[:-1], phase_noise_filter_freqs[1:]]).T,
                          np.stack([phase_noise_filter_amplitude[:-1], phase_noise_filter_amplitude[1:]]).T,
                          #weight=[1, 1, 0.1, 0.00001],
                          fs = config['GENERATE']['sample_rate'])

In [ ]:
fig, ax = plt.subplots()
w, h = scipy.signal.freqz(taps, fs=config['GENERATE']['sample_rate'], worN=100000)

ax.plot(w, 20 * np.log10(np.abs(h)))
ax.set_xscale('log')  # Set x-axis to log scale
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Magnitude [dB]')
ax.set_title('Frequency Response')
ax.scatter(phase_noise_filter_freqs, phase_noise_filter_power_db, color='red')
ax.set_xlim([1, 10e6])
ax.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate white noise
sample_rate = 1000  # Sample rate in Hz
duration = 1  # Duration in seconds
num_samples = sample_rate * duration
white_noise = np.random.randn(num_samples)

# Apply power-law filter to convert to 1/f noise
frequencies = np.fft.fftfreq(num_samples, d=1/sample_rate)
amplitudes = 1 / np.abs(frequencies)
filtered_noise = np.fft.ifft(np.fft.fft(white_noise) * amplitudes).real

# Plot the frequency response
plt.figure()
plt.magnitude_spectrum(filtered_noise, Fs=sample_rate, scale='dB')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
plt.title('Frequency Response')
plt.grid(True)
plt.show()


In [ ]:
config['GENERATE']['sample_rate'] / 1e6

In [ ]:
tmp = cn.powerlaw_psd_gaussian(2, len(sig))
y = tmp
plot_signal(ts, y, one_sided=True, log_scale=True)

In [ ]:
beta = 1 # the exponent
samples = 2**18 # number of samples to generate
y = cn.powerlaw_psd_gaussian(beta, samples)

# optionally plot the Power Spectral Density with Matplotlib
from matplotlib import mlab
from matplotlib import pylab as plt
#s, f = mlab.psd(y, NFFT=2**13)
f, s = scipy.signal.welch(y)
#plt.loglog(f,s)
plt.semilogx(f, 20*np.log10(np.abs(s)))
plt.grid(True)
plt.show()

In [ ]:
plot_signal(np.linspace(0,1,len(y)), y)

In [ ]:
# Generate phase noise samples from sum of 1/f^2 , 1/f, and white noise
# Convert phase noise samples to time jitter by (1/f_adc) * ph / (2*pi)
# Generate chirp phase profiles according to time jitter